In [108]:
import keras
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [5]:
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [7]:
print(housing["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Creating a sequential model

In [29]:
model = Sequential(name="Price_Predictor_Mark_I")
model.add(Dense(30, activation="relu", input_shape=X_train.shape[1:]))
model.add(Dense(1))

In [30]:
model.summary()

Model: "Price_Predictor_Mark_I"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                270       
                                                                 
 dense_4 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301 (1.18 KB)
Trainable params: 301 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.compile(
    loss="mean_squared_error",
    optimizer="sgd",
)

In [32]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)

Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7748 - val_loss: 100709.4062
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5315 - val_loss: 65402.8594
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5002 - val_loss: 48386.7148
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4688 - val_loss: 29146.4668
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6517 - val_loss: 86250.3516
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4568 - val_loss: 64945.8086
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4426 - val_loss: 54597.2578
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4373 - val_loss: 63719.7852
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4288 - val_loss: 76974.3359
Epoch 10/20
363/363 [==============================] - 1s 2ms/s

In [33]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 139605.9844


139605.984375

In [34]:
X_new = X_test[:4]
predicted_prices = model.predict(X_new)

1/1 [==============================] - 0s 45ms/step


In [35]:
predicted_prices

array([[255.69049],
       [344.29218],
       [447.39035],
       [134.25182]], dtype=float32)

In [36]:
y_test[:4]

array([1.335, 3.481, 0.707, 0.55 ])

## California housing : Functional Modelling (Wide & Deep Model)

In [41]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden_1 = keras.layers.Dense(30, activation="relu")(input_)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)

concat = keras.layers.Concatenate()([hidden_1, hidden_2])
output = keras.layers.Dense(1)(concat)

model = keras.Model(inputs=[input_], outputs=[output], name="Price_Predictor_Mark_II")

In [42]:
model.summary()

Model: "Price_Predictor_Mark_II"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 dense_11 (Dense)            (None, 30)                   270       ['input_3[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 30)                   930       ['dense_11[0][0]']            
                                                                                                  
 concatenate_2 (Concatenate  (None, 60)                   0         ['dense_11[0][0]',            
 )                                                                   'dense_

## Creating a Wide and deep model with multiple inputs

In [44]:
input_a = keras.layers.Input(shape=[5], name="wide_path")

input_b = keras.layers.Input(shape=[6], name="deep_path")
hidden_1 = keras.layers.Dense(30, activation="relu")(input_b)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)

concatenate = keras.layers.Concatenate()([hidden_2, input_a])
output = keras.layers.Dense(1)(concatenate)

model_1 = keras.Model(
    inputs=[input_a, input_b], outputs=[output], name="Price_Predictor_Mark_III"
)

In [45]:
model_1.summary()

Model: "Price_Predictor_Mark_III"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 deep_path (InputLayer)      [(None, 6)]                  0         []                            
                                                                                                  
 dense_17 (Dense)            (None, 30)                   210       ['deep_path[0][0]']           
                                                                                                  
 dense_18 (Dense)            (None, 30)                   930       ['dense_17[0][0]']            
                                                                                                  
 wide_path (InputLayer)      [(None, 5)]                  0         []                            
                                                                           

In [53]:
model_1.compile(
    loss="mse",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
)

In [48]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [50]:
X_train_B.shape, X_train_A.shape

((11610, 6), (11610, 5))

In [54]:
history = model_1.fit(
    (X_train_A, X_train_B),
    y_train,
    epochs=20,
    validation_data=((X_valid_A, X_valid_B), y_valid),
)

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 2.2644 - val_loss: 150593.5781
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7723 - val_loss: 147894.0312
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6375 - val_loss: 175364.0000
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5930 - val_loss: 149416.1562
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5676 - val_loss: 137080.3438
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5509 - val_loss: 121060.1016
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5391 - val_loss: 98145.3828
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5300 - val_loss: 91904.6641
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5238 - val_loss: 76171.9219
Epoch 10/20
363/363 [==============================] - 1s 

In [56]:
mse_test = model_1.evaluate((X_test_A, X_test_B), y_test)
y_pred = model_1.predict((X_new_A, X_new_B))

1/1 [==============================] - 0s 57ms/step


In [57]:
y_pred

array([[135.50758],
       [175.32347],
       [221.04468]], dtype=float32)

In [58]:
y_test[:3]

array([1.335, 3.481, 0.707])

## Multi-output Neural Network : So that some part of neural network learns useful stuff on its own

In [64]:
input_a = keras.layers.Input(shape=[5], name="wide_path")

input_b = keras.layers.Input(shape=[6], name="deep_path")
hidden_1 = keras.layers.Dense(30, activation="relu")(input_b)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)

concatenate = keras.layers.Concatenate()([hidden_2, input_a])

output = keras.layers.Dense(1, name="main_output")(concatenate)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden_2)

model_2 = keras.Model(
    inputs=[input_a, input_b],
    outputs=[output, aux_output],
    name="Price_Predictor_Mark_IV",
)

In [65]:
model_2.summary()

Model: "Price_Predictor_Mark_IV"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 deep_path (InputLayer)      [(None, 6)]                  0         []                            
                                                                                                  
 dense_24 (Dense)            (None, 30)                   210       ['deep_path[0][0]']           
                                                                                                  
 dense_25 (Dense)            (None, 30)                   930       ['dense_24[0][0]']            
                                                                                                  
 wide_path (InputLayer)      [(None, 5)]                  0         []                            
                                                                            

In [66]:
"""
In case of multiple outputs, each output will need its own loss function.
Therefore, when we compile the model, we should pass a list of losses.
"""
model_2.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [67]:
history = model_2.fit(
    [X_train_A, X_train_B],
    [y_train, y_train],
    epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]),
)

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 0.9116 - main_output_loss: 0.8026 - aux_output_loss: 1.8921 - val_loss: 245933.7031 - val_main_output_loss: 234658.2812 - val_aux_output_loss: 347413.8750
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5936 - main_output_loss: 0.5262 - aux_output_loss: 1.1998 - val_loss: 105676.2344 - val_main_output_loss: 93927.7969 - val_aux_output_loss: 211412.5156
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5374 - main_output_loss: 0.4883 - aux_output_loss: 0.9794 - val_loss: 49044.5117 - val_main_output_loss: 37850.4297 - val_aux_output_loss: 149791.3125
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5102 - main_output_loss: 0.4726 - aux_output_loss: 0.8482 - val_loss: 67551.1406 - val_main_output_loss: 58187.6875 - val_aux_output_loss: 151822.4688
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4995 - main_o

In [69]:
total_loss, main_loss, aux_loss = model_2.evaluate(
    [X_test_A, X_test_B], [y_test, y_test]
)

162/162 [==============================] - 0s 2ms/step - loss: 60838.1133 - main_output_loss: 53631.9570 - aux_output_loss: 125693.8047


In [70]:
print(total_loss, main_loss, aux_loss)

60838.11328125 53631.95703125 125693.8046875


In [72]:
y_pred_main, y_pred_aux = model_2.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 60ms/step


## Using subclassing API to build a model

In [78]:
"""
Simply subclass the Model class, create the layers you need in the
constructor, and use them to perform the computations you want in the
call() method
"""


class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)  # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output


model_3 = WideAndDeepModel(name="Price_Predictor_Mark_V")

In [81]:
model_3.summary()

Model: "Price_Predictor_Mark_V"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            multiple                  210       
                                                                 
 dense_31 (Dense)            multiple                  930       
                                                                 
 dense_32 (Dense)            multiple                  36        
                                                                 
 dense_33 (Dense)            multiple                  31        
                                                                 
Total params: 1207 (4.71 KB)
Trainable params: 1207 (4.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
model_3.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [80]:
history = model_3.fit(
    [X_train_A, X_train_B],
    [y_train, y_train],
    epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]),
)

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 1.0452 - output_1_loss: 0.9058 - output_2_loss: 2.3002 - val_loss: 30350.7363 - val_output_1_loss: 3210.9172 - val_output_2_loss: 274609.1250
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6779 - output_1_loss: 0.6390 - output_2_loss: 1.0273 - val_loss: 51001.9805 - val_output_1_loss: 37958.7656 - val_output_2_loss: 168390.7812
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.9306 - output_1_loss: 0.9223 - output_2_loss: 1.0054 - val_loss: 23739.5234 - val_output_1_loss: 21112.2734 - val_output_2_loss: 47384.8008
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5187 - output_1_loss: 0.4828 - output_2_loss: 0.8419 - val_loss: 20895.3047 - val_output_1_loss: 18627.8262 - val_output_2_loss: 41302.5898
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4993 - output_1_loss: 0.4685 - output_2_loss: 0.7765 - val_

In [82]:
total_loss, main_loss, aux_loss = model_3.evaluate(
    [X_test_A, X_test_B], [y_test, y_test]
)

162/162 [==============================] - 0s 2ms/step - loss: 52269.1836 - output_1_loss: 43123.2461 - output_2_loss: 134582.5625


In [83]:
print(total_loss, main_loss, aux_loss)

52269.18359375 43123.24609375 134582.5625


In [84]:
y_pred_main, y_pred_aux = model_3.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 56ms/step


In [85]:
print(y_pred_main, y_pred_aux)

[[177.298  ]
 [195.011  ]
 [220.70209]] [[276.64145]
 [336.71625]
 [413.8453 ]]


In [86]:
y_test[:3]

array([1.335, 3.481, 0.707])

## Saving and restoring a model

In [88]:
"""Keras will use the HDF5 format to save both the model’s architecture
(including every layer’s hyperparameters) and the values of all the model
parameters for every layer (e.g., connection weights and biases). It also
saves the optimizer (including its hyperparameters and any state it may
have)."""
model_2.save("keras_functional_model.h5")

SyntaxError: incomplete input (2861843330.py, line 1)

In [89]:
model_2 = keras.models.load_model("keras_functional_model.h5")

In [90]:
"""
For subclassing API, this will unfortunately not work. You can use save_weights() and
load_weights() to at least save and restore the model parameters, but you will
need to save and restore everything else yourself.
"""

'\nFor subclassing API, this will unfortunately not work. You can use save_weights() and\nload_weights() to at least save and restore the model parameters, but you will\nneed to save and restore everything else yourself.\n'

## Using Callbacks to save model at checkpoints while fitting the data

In [95]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "my_keras_model_saved_using_callbacks.h5",
    save_best_only=True  # Only saves the model when performance on validation set is best so far,
    # can be used for Early Stopping
)
history = model_2.fit(
    [X_train_A, X_train_B],
    [y_train, y_train],
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]),
    epochs=20,
    callbacks=[checkpoint_cb],
)

Epoch 1/20
363/363 [==============================] - 1s 4ms/step - loss: 0.3454 - main_output_loss: 0.3373 - aux_output_loss: 0.4183 - val_loss: 137497.5938 - val_main_output_loss: 125242.2031 - val_aux_output_loss: 247796.0625
Epoch 2/20
 57/363 [===>..........................] - ETA: 0s - loss: 0.3353 - main_output_loss: 0.3281 - aux_output_loss: 0.4001

/home/ares/Softwares/PythonVenv/py3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


363/363 [==============================] - 1s 3ms/step - loss: 0.3399 - main_output_loss: 0.3317 - aux_output_loss: 0.4136 - val_loss: 180117.5156 - val_main_output_loss: 164551.6562 - val_aux_output_loss: 320210.0625
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3394 - main_output_loss: 0.3312 - aux_output_loss: 0.4134 - val_loss: 191713.3125 - val_main_output_loss: 175319.5781 - val_aux_output_loss: 339256.9688
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3396 - main_output_loss: 0.3311 - aux_output_loss: 0.4161 - val_loss: 124381.6016 - val_main_output_loss: 111889.5469 - val_aux_output_loss: 236810.0000
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3391 - main_output_loss: 0.3312 - aux_output_loss: 0.4110 - val_loss: 103906.3672 - val_main_output_loss: 92269.5469 - val_aux_output_loss: 208637.6719
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3374 - main_output_l

## Searching for optimum hyperparameters using RandomisedSearchCV [NOT WORKING!]: TRY KERAS TUNER LATER

In [119]:
def build_model(n_hidden=1, n_neurons=30, optimizer_learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=optimizer_learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [120]:
# Creating a keras regressor on the above defined model function
keras_reg = KerasRegressor(build_model)

In [121]:
keras_reg.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.2217 - val_loss: 105221.4766
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6189 - val_loss: 127731.5078
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5648 - val_loss: 81686.0234
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5317 - val_loss: 73249.5938
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5078 - val_loss: 65894.3906
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4909 - val_loss: 51382.8008
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4792 - val_loss: 49408.7383
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4713 - val_loss: 54753.3477
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4656 - val_loss: 55384.1289
Epoch 10/100
363/363 [==============================]

In [125]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons": np.arange(1, 100),
    "optimizer_learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)

ValueError: Invalid parameter n_neurons for estimator KerasRegressor.
This issue can likely be resolved by setting this parameter in the KerasRegressor constructor:
`KerasRegressor(n_neurons=57)`
Check the list of available parameters with `estimator.get_params().keys()`

In [123]:
keras_reg.get_params()+

{'model': <function __main__.build_model(n_hidden=1, n_neurons=30, optimizer_learning_rate=0.003, input_shape=[8])>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': 'rmsprop',
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1}